In [ ]:
import torch, train, random, importlib, torchvision

import dataset
import utils
import net

from PIL import ImageDraw
from IPython.display import display

torch.set_printoptions(precision=3, linewidth=300)

In [ ]:
importlib.reload(utils)
importlib.reload(dataset)

cv = utils.Converter()
d = dataset.RisikoDataset(dataset_dir='Datasets/Generated_Dataset/train', cv=cv, train_mode=False)

In [ ]:
model = torch.load('weights/last_model').to('cpu')
model.eval()

In [ ]:
importlib.reload(utils)

rnd_idx = random.randint(0, len(d))
img_tensor, target = d.__getitem__(idx=rnd_idx, return_basic_labels=True)

netout = model.forward(img_tensor.unsqueeze(0)).squeeze(0)
preds_labels = cv.convert_netout_to_basic_labels(netout=netout, probability_threshold=0.4, max_iou=0.5, apply_nms=True)
cv.convert_labels_from_relative_to_absolute_values(preds_labels[:,1:], use_netin_shape=True)

img =  torch.multiply(img_tensor, 256).type(torch.uint8)
to_pil = torchvision.transforms.ToPILImage()
img = to_pil(img)

colors = [    "blue",    "red",    "yellow",    "purple",    "black",    "green"]

boxes = preds_labels
print(boxes.shape[0])

if (boxes.shape[0] != 0):

    if len(boxes.shape) == 1:
        boxes = boxes.unsqueeze(1)

    box_coords = boxes[:, 2:]
    # torch.multiply(box_coords, torch.tensor([img.width, img.height, img.width, img.height], dtype=torch.float32), out=box_coords)
    # print(box_coords)

    img_draw = ImageDraw.Draw(img)

    for i in range(box_coords.shape[0]):

        x0 = int(box_coords[i,0] - box_coords[i,2] / 2)
        x1 = int(box_coords[i,0] + box_coords[i,2] / 2)
        y0 = int(box_coords[i,1] - box_coords[i,3] / 2)
        y1 = int(box_coords[i,1] + box_coords[i,3] / 2)

        img_draw.rectangle([x0, y0, x1, y1], outline=colors[int(boxes[i,1])%6])

display(img)